In [9]:
import os
from datetime import datetime
from typing import Type, Any

from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper

from duckduckgo_search import DDGS


from pydantic import BaseModel, Field

openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    api_key=openai_api_key
)

class WikipediaSearchArgsSchema(BaseModel):
    query: str = Field(description="The query to search on Wikipedia")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Searches a query on Wikipedia and returns the summary"
    args_schema: Type[WikipediaSearchArgsSchema] = WikipediaSearchArgsSchema

    def _run(self, query):
        wiki_api_wrapper = WikipediaAPIWrapper()
        runner = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
        return runner.run(query)


class DuckDuckGoSearchArgsSchema(BaseModel):
    query: str = Field(description="The query to search on DuckDuckGo")

class DuckDuckGoSearchTool(WikipediaSearchTool):
    name = "DuckDuckGoSearchTool"
    description = "Searches a query on DuckDuckGo and returns the summary"
    args_schema: Type[DuckDuckGoSearchArgsSchema] = DuckDuckGoSearchArgsSchema

    def _run(self, query):
        ddg = DDGS()
        return ddg.text(query)

        
class GenerateTextFileTypeArgsSchema(BaseModel):
    content: str = Field(description="The content to write to the file")

class GenerateTextFileTool(BaseTool):
    name = "GenerateTextFileTool"
    description = "Generates a text file with the given content"
    args_schema: Type[GenerateTextFileTypeArgsSchema] = GenerateTextFileTypeArgsSchema

    def _run(self, content):
        current_date = datetime.now().strftime("%y%m%d")
        with open(f"./files/result_{current_date}.txt", "w+", encoding="utf-8") as f:
            f.write(content)

        

agent = initialize_agent(
    llm = llm,
    verbose = True,
    agent = AgentType.OPENAI_FUNCTIONS,
    tools = [
        DuckDuckGoSearchTool(),
        WikipediaSearchTool(),
        GenerateTextFileTool(),
    ],
    # toweringcloud 님 코드 참조
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
                you are an agent that can search information on the internet and save the result into a file.
                You collect and compare the received data, then choose the best option and create a file with it.
                please make sure to aware of amount of words in the file under 1000 words. and wrie the source in the file.
            """
        )
    },
)



In [10]:

agent.invoke("Research about the XZ backdoor.")



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`


[{'title': 'XZ Utils backdoor | Wikipedia', 'href': 'https://en.wikipedia.org/wiki/XZ_Utils_backdoor', 'body': 'In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". [b] [2] The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system.The issue has been given the Common Vulnerabilities and Exposures number CVE ...'}, {'title': 'The XZ Backdoor: Everything You Need to Know | WIRED', 'href': 'https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/', 'body': 'What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ...'}, {'title': "The Mys

{'input': 'Research about the XZ backdoor.',
 'output': 'I have researched the XZ backdoor and compiled the information into a text file. Here is a summary of the findings:\n\n---\n\n# XZ Backdoor Overview\n\nIn February 2024, a malicious backdoor was introduced to the Linux utility XZ within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". This backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code on the affected Linux systems. The vulnerability has been assigned the Common Vulnerabilities and Exposures (CVE) number CVE-2024-3094 and has a CVSS score of 10.0, indicating a critical severity level.\n\n## Impact and Discovery\n\nThe XZ utility is commonly included in most Linux distributions, but at the time of the backdoor\'s discovery, the compromised versions had not been widely deployed in production systems. They were primarily present in development versions of major distributions. The backdoor was disco